In [1]:
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport

import time
from datetime import timedelta
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [2]:
# # разархивируем датасет что считать его ниже
# # pip install py7zr
# import py7zr
# with py7zr.SevenZipFile('1. Исходные данные и результаты прошлого года.7z', mode='r') as z:
#     z.extractall()

# statistics

In [3]:
statistics = pd.read_csv('statistics-07-20.csv', delimiter=";")
statistics.info()
statistics[:3]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4189915 entries, 0 to 4189914
Data columns (total 21 columns):
 #   Column            Dtype  
---  ------            -----  
 0   index_train       float64
 1   length            float64
 2   car_number        int64  
 3   destination_esr   float64
 4   adm               float64
 5   danger            float64
 6   gruz              float64
 7   loaded            float64
 8   operation_car     float64
 9   operation_date    object 
 10  operation_st_esr  float64
 11  operation_st_id   float64
 12  operation_train   float64
 13  receiver          float64
 14  rodvag            float64
 15  rod_train         float64
 16  sender            float64
 17  ssp_station_esr   float64
 18  ssp_station_id    float64
 19  tare_weight       float64
 20  weight_brutto     float64
dtypes: float64(19), int64(1), object(1)
memory usage: 671.3+ MB


,index_train,length,car_number,destination_esr,adm,danger,gruz,loaded,operation_car,operation_date,...,operation_st_id,operation_train,receiver,rodvag,rod_train,sender,ssp_station_esr,ssp_station_id,tare_weight,weight_brutto
0,NaN,NaN,62827035,984700.0,NaN,NaN,NaN,2.0,81.0,2020-07-16 03:40:00,...,2.000039e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,1.0,62827035,862201.0,20.0,NaN,421034.0,NaN,18.0,2020-07-16 14:10:00,...,2.002026e+09,NaN,93149858.0,60.0,NaN,68398528.0,NaN,NaN,249.0,NaN
2,NaN,NaN,62827316,863007.0,NaN,NaN,NaN,2.0,80.0,2020-07-16 14:50:00,...,2.001933e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
statistics.columns

Index(['index_train', 'length', 'car_number', 'destination_esr', 'adm',
       'danger', 'gruz', 'loaded', 'operation_car', 'operation_date',
       'operation_st_esr', 'operation_st_id', 'operation_train', 'receiver',
       'rodvag', 'rod_train', 'sender', 'ssp_station_esr', 'ssp_station_id',
       'tare_weight', 'weight_brutto'],
      dtype='object')

Обозначение столбцов в statistics-07-20.csv

index_train - Индекс поезда
length - Длина вагона
car_number - Номер вагона
destination_esr - ЕСР станции назначения вагона 
adm - Код АСОУП администрации-собственника вагона
danger - Признак опасного груза в вагоне. 1    - в вагоне опасный груз,  null  - иначе  
gruz - Код груза 
loaded - Признак груженого вагона. 1 -  груженый, 2- порожний
operation_car - Код операции ВМ  АСОУП
operation_date - Дата операции
operation_st_esr - ЕСР станции операции
operation_st_id - ID станции операции
operation_train - Код операции ПМ  АСОУП
receiver - ОКПО грузополучателя
rodvag - Род вагона АСОУП  (учетного)
rod_train - Род поезда АСОУП
sender - ОКПО грузоотправителя
ssp_station_esr - ЕСР станция ССП
ssp_station_id - id станции ССП
tare_weight - Вес тары вагона  (ц)
weight_brutto - weight_brutto

In [5]:
# # посмотрим варианты кодов операций
# statistics_vc = statistics.operation_car.value_counts().rename_axis('unique_values').reset_index(name='counts')
# statistics_vc['statistics'] = 'statistics_in_operation_car'
# statistics_vc

In [6]:
# statistics_vcc = statistics.operation_train.value_counts().rename_axis('unique_values').reset_index(name='counts')
# statistics_vcc['statistics'] = 'statistics_in_operation_car'
# statistics_vcc

In [7]:
# оставляем нужные столбцы
statistics = statistics[['operation_date', 'operation_st_esr', 'ssp_station_esr', 'car_number', 'operation_car', 'operation_train', 'tare_weight', 'weight_brutto']]

макс, пускай возмем за корректную станцию из operation_st_esr и ssp_station_esr -> operation_st_esr. убираем ssp_station_esr

In [8]:
# оставляем нужные столбцы
statistics = statistics[['operation_date', 'operation_st_esr', 'car_number', 'operation_car']]
# убираем пропуски по станциям
statistics = statistics[~(statistics.operation_st_esr.isna())]
display(statistics[:8])
display(statistics.info())
display(statistics.shape)

,operation_date,operation_st_esr,car_number,operation_car
0,2020-07-16 03:40:00,984502.0,62827035,81.0
1,2020-07-16 14:10:00,984700.0,62827035,18.0
2,2020-07-16 14:50:00,863007.0,62827316,80.0
3,2020-07-16 14:16:00,862108.0,62827381,80.0
4,2020-07-16 15:15:00,967808.0,62845730,19.0
5,2020-07-16 06:30:00,862409.0,62826326,10.0
6,2020-07-16 08:28:00,862409.0,62826326,4.0
7,2020-07-15 19:59:00,887800.0,62826987,4.0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4189317 entries, 0 to 4189914
Data columns (total 4 columns):
 #   Column            Dtype  
---  ------            -----  
 0   operation_date    object 
 1   operation_st_esr  float64
 2   car_number        int64  
 3   operation_car     float64
dtypes: float64(2), int64(1), object(1)
memory usage: 159.8+ MB


None

(4189317, 4)

In [9]:
# data = statistics[statistics.car_number == 55864821]
# data = data.sort_values('operation_date')
# data['operation_date'] = pd.to_datetime(data['operation_date']) # , format='%Y-%m-%d %HH:%mm'

In [10]:
# import time.
# start_time = time. time()

# statistics = statistics[statistics['operation_st_esr'].notna()]

# print("--- %s seconds ---" % (time. time() - start_time))
# statistics

In [11]:
# profile = ProfileReport(statistics, title="statistics Profiling Report")
# profile

# podvod_pogruzka

In [12]:
# podvod = pd.read_excel('Под вод вагонов на Полотняный завод.xlsx')
# podvod_pogruzka = pd.read_excel(podvod, 'Погрузка')
# podvod_gruzenih = pd.read_excel(podvod, 'Подвод груженых')

In [13]:
podvod_pogruzka = pd.read_excel('Подвод вагонов на Полотняный завод.xlsx', sheet_name = 'Погрузка')
podvod_pogruzka.info()
podvod_pogruzka[:3]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2445 entries, 0 to 2444
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   kop_vmd    2445 non-null   int64         
 1   date_op    2445 non-null   datetime64[ns]
 2   nom_vagon  2445 non-null   int64         
 3   stan_nazn  2445 non-null   int64         
 4   ves_grz    2445 non-null   int64         
dtypes: datetime64[ns](1), int64(4)
memory usage: 95.6 KB


,kop_vmd,date_op,nom_vagon,stan_nazn,ves_grz
0,11,2021-08-01 06:17:00,50501485,35809,63183
1,11,2021-08-01 06:17:00,73765281,35809,63263
2,11,2021-08-01 06:17:00,51186443,35809,63804


In [14]:
# # посмотрим варианты кодов операций
# podvod_pogruzka_vc = podvod_pogruzka.kop_vmd.value_counts().rename_axis('unique_values').reset_index(name='counts')
# podvod_pogruzka_vc['table'] = 'podvod_pogruzka'
# podvod_pogruzka_vc

In [15]:
podvod_pogruzka = podvod_pogruzka[['date_op', 'stan_nazn', 'nom_vagon', 'kop_vmd']]
podvod_pogruzka

,date_op,stan_nazn,nom_vagon,kop_vmd
0,2021-08-01 06:17:00,35809,50501485,11
1,2021-08-01 06:17:00,35809,73765281,11
2,2021-08-01 06:17:00,35809,51186443,11
3,2021-08-01 06:17:00,35809,73747636,11
4,2021-08-01 06:17:00,35809,50730852,11
...,...,...,...,...
2440,2021-08-30 21:17:00,804600,64576754,18
2441,2021-08-30 21:17:00,804600,64576606,18
2442,2021-08-30 21:17:00,804600,64576747,18
2443,2021-08-30 21:21:00,205400,60122918,18


In [16]:
# profile2 = ProfileReport(podvod_pogruzka, title="podvod_pogruzka Profiling Report")
# profile2

# podvod_gruzenih

In [17]:
podvod_gruzenih = pd.read_excel('Подвод вагонов на Полотняный завод.xlsx', sheet_name = 'Подвод груженых')
podvod_gruzenih.info()
podvod_gruzenih[:3]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6142 entries, 0 to 6141
Data columns (total 13 columns):
 #   Column                                        Non-Null Count  Dtype         
---  ------                                        --------------  -----         
 0   Номер вагона                                  4463 non-null   float64       
 1   Код предыдущей грузовой операции              4017 non-null   float64       
 2   Дата предыдущей грузовой операции             4017 non-null   datetime64[ns]
 3   Вес груза (предыдущей грузовой операции)      4017 non-null   float64       
 4   Станция предыдущей грузовой операции          4017 non-null   float64       
 5   Код операции (искл. из поезда) на данной ст.  4463 non-null   float64       
 6   Дата операции на данной ст.                   4463 non-null   datetime64[ns]
 7   Признак порожнего вагона на данной ст.        4463 non-null   float64       
 8   Дата грузовой операции на данной ст.          2473 non-null   dateti

,Номер вагона,Код предыдущей грузовой операции,Дата предыдущей грузовой операции,Вес груза (предыдущей грузовой операции),Станция предыдущей грузовой операции,Код операции (искл. из поезда) на данной ст.,Дата операции на данной ст.,Признак порожнего вагона на данной ст.,Дата грузовой операции на данной ст.,Код грузовой операции на данной ст.,Фильтр,От,До
0,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,44409,44439
1,28035517.0,18.0,2021-08-12 18:01:00,0.0,205608.0,3.0,2021-08-21 03:01:00,1.0,2021-08-24 17:15:00,10.0,1.0,2021-08-01 00:00:00,2021-08-31 00:00:00
2,28071785.0,19.0,2021-07-29 16:24:00,0.0,210108.0,3.0,2021-08-07 19:36:00,1.0,2021-08-09 16:59:00,10.0,1.0,NaN,NaN


In [18]:
# # посмотрим варианты кодов операций
# podvod_gruzenih_vc = podvod_gruzenih['Код предыдущей грузовой операции'].value_counts().rename_axis('unique_values').reset_index(name='counts')
# podvod_gruzenih_vc['table'] = 'podvod_gruzenih'
# podvod_gruzenih_vc

In [19]:
# # обьеденим все коды станций value_counts()
# frames = [statistics_vc, statistics_vcc, podvod_pogruzka_vc, podvod_gruzenih_vc]
# united_vc = pd.concat(frames)
# # united_vc.to_excel("united_vc.xlsx")
# united_vc

In [20]:
# уберем ненужные столбцы
podvod_gruzenih = podvod_gruzenih[['Дата предыдущей грузовой операции', 'Станция предыдущей грузовой операции', 'Номер вагона', 'Код предыдущей грузовой операции']]

In [21]:
# уберем строки где все признаки неизвестны
podvod_gruzenih = podvod_gruzenih.dropna(axis=0, how="all", thresh=None, subset=None, inplace=False)

In [22]:
# уберем строки где дата и станция не известны
podvod_gruzenih = podvod_gruzenih[podvod_gruzenih['Дата предыдущей грузовой операции'].notna()]
podvod_gruzenih = podvod_gruzenih[podvod_gruzenih['Станция предыдущей грузовой операции'].notna()]
podvod_gruzenih

,Дата предыдущей грузовой операции,Станция предыдущей грузовой операции,Номер вагона,Код предыдущей грузовой операции
1,2021-08-12 18:01:00,205608.0,28035517.0,18.0
2,2021-07-29 16:24:00,210108.0,28071785.0,19.0
3,2021-07-30 02:06:00,200500.0,28072015.0,19.0
6,2021-06-04 20:49:00,229706.0,28087146.0,18.0
7,2021-06-03 09:47:00,946801.0,28088029.0,11.0
...,...,...,...,...
4459,2021-06-03 10:21:00,211609.0,95705620.0,18.0
4460,2021-07-15 13:37:00,211609.0,95724126.0,18.0
4461,2021-06-16 10:45:00,211609.0,95729265.0,18.0
4462,2021-06-16 10:48:00,211609.0,95729455.0,18.0


# обьединение

In [23]:
statistics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4189317 entries, 0 to 4189914
Data columns (total 4 columns):
 #   Column            Dtype  
---  ------            -----  
 0   operation_date    object 
 1   operation_st_esr  float64
 2   car_number        int64  
 3   operation_car     float64
dtypes: float64(2), int64(1), object(1)
memory usage: 159.8+ MB


In [24]:
podvod_pogruzka.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2445 entries, 0 to 2444
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date_op    2445 non-null   datetime64[ns]
 1   stan_nazn  2445 non-null   int64         
 2   nom_vagon  2445 non-null   int64         
 3   kop_vmd    2445 non-null   int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 76.5 KB


In [25]:
podvod_gruzenih.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4017 entries, 1 to 4463
Data columns (total 4 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   Дата предыдущей грузовой операции     4017 non-null   datetime64[ns]
 1   Станция предыдущей грузовой операции  4017 non-null   float64       
 2   Номер вагона                          4017 non-null   float64       
 3   Код предыдущей грузовой операции      4017 non-null   float64       
dtypes: datetime64[ns](1), float64(3)
memory usage: 156.9 KB


In [26]:
statistics.columns

Index(['operation_date', 'operation_st_esr', 'car_number', 'operation_car'], dtype='object')

In [27]:
# переименуем столбцы
podvod_pogruzka.columns = ['operation_date', 'operation_st_esr', 'car_number', 'operation_car']
podvod_gruzenih.columns = ['operation_date', 'operation_st_esr', 'car_number', 'operation_car']

In [28]:
# # изменим типы в столбцах
# start_time = time. time()

# statistics.operation_date = statistics.operation_date.apply(pd.to_datetime)
# statistics.operation_st_esr = statistics.operation_st_esr.astype(int)
# statistics.car_number = statistics.car_number.astype(int)

# podvod_pogruzka.operation_st_esr = podvod_pogruzka.operation_st_esr.astype(int)
# podvod_pogruzka.car_number = podvod_pogruzka.car_number.astype(int)

# podvod_gruzenih.operation_st_esr = podvod_gruzenih.operation_st_esr.astype(int)
# podvod_gruzenih.car_number = podvod_gruzenih.car_number.astype(int)

# print("--- %s seconds ---" % (time. time() - start_time))

--- 202.27835726737976 seconds ---


In [29]:
# # обьединяем датасеты
# start_time = time. time()

# frames = [statistics, podvod_pogruzka, podvod_gruzenih]
# data = pd.concat(frames)

# print("--- %s seconds ---" % (time. time() - start_time))
# data.to_csv("data.csv")
# data

--- 0.05893754959106445 seconds ---


,operation_date,operation_st_esr,car_number,operation_car
0,2020-07-16 03:40:00,984502,62827035,81.0
1,2020-07-16 14:10:00,984700,62827035,18.0
2,2020-07-16 14:50:00,863007,62827316,80.0
3,2020-07-16 14:16:00,862108,62827381,80.0
4,2020-07-16 15:15:00,967808,62845730,19.0
...,...,...,...,...
4459,2021-06-03 10:21:00,211609,95705620,18.0
4460,2021-07-15 13:37:00,211609,95724126,18.0
4461,2021-06-16 10:45:00,211609,95729265,18.0
4462,2021-06-16 10:48:00,211609,95729455,18.0


# data

In [41]:
# data.to_csv("data.csv", index=False)

In [2]:
data = pd.read_csv('data.csv')
data = data.drop(data.columns[0], axis=1)

In [3]:
# добавим описание операции
codes = pd.read_excel('Расшифровка операций с вагонами.xlsx')
codes.columns = ['operation_car', 'short_name', 'operation_name']
data = data.merge(codes, on='operation_car', how='left')
data

,operation_date,operation_st_esr,car_number,operation_car,short_name,operation_name
0,2020-07-16 03:40:00,984502,62827035,81.0,ПВПП,ПРИЕМ ВАГОНА С ПОД'ЕЗДНОГО ПУТИ
1,2020-07-16 14:10:00,984700,62827035,18.0,ПГР0,ПОГРУЗКА БЕЗ ЗАЧЕТА В ПОГРУЗКУ
2,2020-07-16 14:50:00,863007,62827316,80.0,СВПП,СДАЧА ВАГОНА НА ПОД'ЕЗДНОЙ ПУТЬ
3,2020-07-16 14:16:00,862108,62827381,80.0,СВПП,СДАЧА ВАГОНА НА ПОД'ЕЗДНОЙ ПУТЬ
4,2020-07-16 15:15:00,967808,62845730,19.0,ПГР9,ПОГРУЗКА БЕЗ ЗАЧЕТА В ПОГРУЗКУ НА ПП
...,...,...,...,...,...,...
4195774,2021-06-03 10:21:00,211609,95705620,18.0,ПГР0,ПОГРУЗКА БЕЗ ЗАЧЕТА В ПОГРУЗКУ
4195775,2021-07-15 13:37:00,211609,95724126,18.0,ПГР0,ПОГРУЗКА БЕЗ ЗАЧЕТА В ПОГРУЗКУ
4195776,2021-06-16 10:45:00,211609,95729265,18.0,ПГР0,ПОГРУЗКА БЕЗ ЗАЧЕТА В ПОГРУЗКУ
4195777,2021-06-16 10:48:00,211609,95729455,18.0,ПГР0,ПОГРУЗКА БЕЗ ЗАЧЕТА В ПОГРУЗКУ


In [4]:
data.operation_st_esr.value_counts()

985906    207099
967600    146427
946801    116951
863007     96308
864207     82794
           ...  
233805         1
592401         1
510907         1
188101         1
18409          1
Name: operation_st_esr, Length: 952, dtype: int64

In [5]:
# оставляем одну станцию 985906
data_station = data[data.operation_st_esr == 985906]
data_station = data_station.sort_values('operation_date')
data_station

,operation_date,operation_st_esr,car_number,operation_car,short_name,operation_name
2847001,2020-07-01 00:18:00,985906,57945560,18.0,ПГР0,ПОГРУЗКА БЕЗ ЗАЧЕТА В ПОГРУЗКУ
2846597,2020-07-01 00:20:00,985906,60154754,18.0,ПГР0,ПОГРУЗКА БЕЗ ЗАЧЕТА В ПОГРУЗКУ
2847913,2020-07-01 01:32:00,985906,56814106,4.0,ВКЛП,ВКЛЮЧЕНИЕ В ПОЕЗД
2841838,2020-07-01 01:32:00,985906,63496475,4.0,ВКЛП,ВКЛЮЧЕНИЕ В ПОЕЗД
2841886,2020-07-01 01:32:00,985906,63481485,4.0,ВКЛП,ВКЛЮЧЕНИЕ В ПОЕЗД
...,...,...,...,...,...,...
2823606,2020-07-31 23:16:00,985906,53836805,21.0,ВЫГ2,ВЫГРУЗКА НА ПУТЯХ КЛИЕНТУРЫ
2830179,2020-07-31 23:16:00,985906,50045392,21.0,ВЫГ2,ВЫГРУЗКА НА ПУТЯХ КЛИЕНТУРЫ
2787290,2020-07-31 23:16:00,985906,55076194,21.0,ВЫГ2,ВЫГРУЗКА НА ПУТЯХ КЛИЕНТУРЫ
2794687,2020-07-31 23:16:00,985906,68054238,21.0,ВЫГ2,ВЫГРУЗКА НА ПУТЯХ КЛИЕНТУРЫ


In [6]:
# to_datetime
start_time = time. time()

data_station['operation_date'] = data_station['operation_date'].apply(pd.to_datetime)

print("--- %s seconds ---" % (time. time() - start_time))

--- 9.927789449691772 seconds ---


In [7]:
# найдем список вагонов где больше одной операции на этой станции
cars_with_more_1_operations = list(data_station.car_number.value_counts()[data_station.car_number.value_counts() > 1].index)

# оставим нужные вагоны где больше одной операции
data_station = data_station[data_station['car_number'].isin(cars_with_more_1_operations)]

In [8]:
# i = 94033792
# data_duration = pd.DataFrame()
# # сортируем по номеру вагона
# car_number_data = data_station[data_station.car_number == i]
# # to_datetime
# car_number_data['operation_date'] = car_number_data['operation_date'].apply(pd.to_datetime)
# # добавим время окончания операции операции
# car_number_data['fin_operation_date'] = car_number_data['operation_date'].shift(-1)
# #     # узнаем длительность операции
# car_number_data['operation_duration'] = (
#     car_number_data['fin_operation_date'] - car_number_data['operation_date']) / timedelta(minutes=1)
# #     # уберем пустые строки (аномальные значения операций)
# car_number_data = car_number_data[car_number_data['operation_car'].notna()]
# car_number_data['operation_duration'] = car_number_data['operation_duration'].astype(int)
# data_duration = data_duration.append(car_number_data)

In [9]:
data_station

,operation_date,operation_st_esr,car_number,operation_car,short_name,operation_name
2847001,2020-07-01 00:18:00,985906,57945560,18.0,ПГР0,ПОГРУЗКА БЕЗ ЗАЧЕТА В ПОГРУЗКУ
2846597,2020-07-01 00:20:00,985906,60154754,18.0,ПГР0,ПОГРУЗКА БЕЗ ЗАЧЕТА В ПОГРУЗКУ
2846076,2020-07-01 01:32:00,985906,59079160,4.0,ВКЛП,ВКЛЮЧЕНИЕ В ПОЕЗД
2850687,2020-07-01 01:32:00,985906,53129318,4.0,ВКЛП,ВКЛЮЧЕНИЕ В ПОЕЗД
2843260,2020-07-01 01:32:00,985906,62681861,4.0,ВКЛП,ВКЛЮЧЕНИЕ В ПОЕЗД
...,...,...,...,...,...,...
2803700,2020-07-31 23:16:00,985906,62593512,21.0,ВЫГ2,ВЫГРУЗКА НА ПУТЯХ КЛИЕНТУРЫ
2822656,2020-07-31 23:16:00,985906,54130588,21.0,ВЫГ2,ВЫГРУЗКА НА ПУТЯХ КЛИЕНТУРЫ
2823606,2020-07-31 23:16:00,985906,53836805,21.0,ВЫГ2,ВЫГРУЗКА НА ПУТЯХ КЛИЕНТУРЫ
2830179,2020-07-31 23:16:00,985906,50045392,21.0,ВЫГ2,ВЫГРУЗКА НА ПУТЯХ КЛИЕНТУРЫ


In [10]:
start_time = time. time()
# # вагон с самым большим количеством операций на этой станции 94033792
# df = df[df.car_number == 94033792]

data_duration = pd.DataFrame()
# цикл для определения длительности операций
for i in list(data_station.car_number.unique()):

    # сортируем по номеру вагона
    car_number_data = data_station[data_station.car_number == i]
    # to_datetime
    car_number_data['operation_date'] = car_number_data['operation_date'].apply(pd.to_datetime)
    # добавим время окончания операции операции
    car_number_data['fin_operation_date'] = car_number_data['operation_date'].shift(-1)
    car_number_data['next_operation_car'] = car_number_data['operation_car'].shift(-1)
    car_number_data['next_operation_name'] = car_number_data['operation_name'].shift(-1)
    # узнаем длительность операции
    car_number_data['operation_duration'] = (
        car_number_data['fin_operation_date'] - car_number_data['operation_date']) / timedelta(minutes=1)
    # уберем пустые строки (аномальные значения операций)
#     car_number_data = car_number_data[car_number_data['operation_car'].notna()]
#     car_number_data['operation_duration'] = car_number_data['operation_duration'].astype(int)
    data_duration = data_duration.append(car_number_data)

print("--- %s seconds ---" % (time. time() - start_time))

--- 598.8874878883362 seconds ---


In [11]:
# data_duration.to_csv("data_duration.csv", index=False)

In [620]:
data_duration = pd.read_csv('data_duration.csv')

In [401]:
# избавимся от пропусков
data_duration = data_duration[data_duration['operation_car'].notna()]

data_duration = data_duration[data_duration['operation_duration'].notna()]

data_duration['next_operation_car'] = data_duration['next_operation_car'].fillna(0)
data_duration['next_operation_name'] = data_duration['next_operation_name'].fillna(0)

In [402]:
# так же изза человеческой ошибки дублируются одинаковые операции
data_duration = data_duration[data_duration['operation_car'] != data_duration['next_operation_car']]

# убираем выбросы, которые считаются комбинацией (ВКЛЮЧЕНИЕ В ПОЕЗД и ИСКЛЮЧЕНИЕ ИЗ ПОЕЗДА) 
# и (ВКЛЮЧЕНИЕ В ПОЕЗД и ВЫГРУЗКА НА ПУТЯХ КЛИЕНТУРЫ)
# и (ВКЛЮЧЕНИЕ В ПОЕЗД и ПРИЕМ ВАГОНА С ПОД'ЕЗДНОГО ПУТИ)

op_ind = list(data_duration[data_duration.operation_car.isin([4])].index)
nop_ind = list(data_duration[data_duration.next_operation_car.isin([3, 21, 81, 80])].index)
op_ind_as_set = set(op_ind)
intersection = list(op_ind_as_set.intersection(nop_ind))
data_duration = data_duration.drop(intersection)


# data_duration[['fin_operation_date',
#              'next_operation_car',
#              'next_operation_name',
#              'operation_duration']] = data_duration[['fin_operation_date',
#                                                      'next_operation_car',
#                                                      'next_operation_name',
#                                                      'operation_duration']].fillna(0)

In [403]:
# ноль минут операция скорее всего из-за человеческой ошибка
data_duration = data_duration[data_duration['operation_duration'] > 0]

In [404]:
# data_duration[data_duration['operation_duration'] == 8]['operation_name'].value_counts()

In [405]:
Q1 =  data_duration['operation_duration'].quantile(0.25)
Q3 = data_duration['operation_duration'].quantile(0.75)
IQR = Q3 - Q1
data_duration = data_duration[data_duration['operation_duration'] < Q3+1.5*IQR]

# Узнать вероятности 

In [669]:
data_duration = pd.read_csv('data_duration.csv')

In [670]:
data_duration

,operation_date,operation_st_esr,car_number,operation_car,short_name,operation_name,fin_operation_date,next_operation_car,next_operation_name,operation_duration
0,2020-07-01 00:18:00,985906,57945560,18.0,ПГР0,ПОГРУЗКА БЕЗ ЗАЧЕТА В ПОГРУЗКУ,2020-07-01 01:32:00,4.0,ВКЛЮЧЕНИЕ В ПОЕЗД,74.0
1,2020-07-01 01:32:00,985906,57945560,4.0,ВКЛП,ВКЛЮЧЕНИЕ В ПОЕЗД,NaN,NaN,NaN,NaN
2,2020-07-01 00:20:00,985906,60154754,18.0,ПГР0,ПОГРУЗКА БЕЗ ЗАЧЕТА В ПОГРУЗКУ,2020-07-01 01:32:00,4.0,ВКЛЮЧЕНИЕ В ПОЕЗД,72.0
3,2020-07-01 01:32:00,985906,60154754,4.0,ВКЛП,ВКЛЮЧЕНИЕ В ПОЕЗД,NaN,NaN,NaN,NaN
4,2020-07-01 01:32:00,985906,59079160,4.0,ВКЛП,ВКЛЮЧЕНИЕ В ПОЕЗД,2020-07-20 06:00:00,3.0,ИСКЛЮЧЕНИЕ ИЗ ПОЕЗДА,27628.0
...,...,...,...,...,...,...,...,...,...,...
204314,2020-07-31 21:23:00,985906,54157193,80.0,СВПП,СДАЧА ВАГОНА НА ПОД'ЕЗДНОЙ ПУТЬ,NaN,NaN,NaN,NaN
204315,2020-07-31 21:20:00,985906,94858685,81.0,ПВПП,ПРИЕМ ВАГОНА С ПОД'ЕЗДНОГО ПУТИ,2020-07-31 21:23:00,80.0,СДАЧА ВАГОНА НА ПОД'ЕЗДНОЙ ПУТЬ,3.0
204316,2020-07-31 21:23:00,985906,94858685,80.0,СВПП,СДАЧА ВАГОНА НА ПОД'ЕЗДНОЙ ПУТЬ,NaN,NaN,NaN,NaN
204317,2020-07-31 21:20:00,985906,59270199,81.0,ПВПП,ПРИЕМ ВАГОНА С ПОД'ЕЗДНОГО ПУТИ,2020-07-31 21:23:00,80.0,СДАЧА ВАГОНА НА ПОД'ЕЗДНОЙ ПУТЬ,3.0


In [671]:
# избавимся от пропусков
data_duration = data_duration[data_duration['operation_car'].notna()]

data_duration = data_duration[data_duration['operation_duration'].notna()]

data_duration['next_operation_car'] = data_duration['next_operation_car'].fillna(0)
data_duration['next_operation_name'] = data_duration['next_operation_name'].fillna(0)

In [672]:
# так же изза человеческой ошибки дублируются одинаковые операции
data_duration = data_duration[data_duration['operation_car'] != data_duration['next_operation_car']]

# убираем выбросы, которые считаются комбинацией (ВКЛЮЧЕНИЕ В ПОЕЗД и ИСКЛЮЧЕНИЕ ИЗ ПОЕЗДА) 
# и (ВКЛЮЧЕНИЕ В ПОЕЗД и ВЫГРУЗКА НА ПУТЯХ КЛИЕНТУРЫ)
# и (ВКЛЮЧЕНИЕ В ПОЕЗД и ПРИЕМ ВАГОНА С ПОД'ЕЗДНОГО ПУТИ)

op_ind = list(data_duration[data_duration.operation_car.isin([4])].index)
nop_ind = list(data_duration[data_duration.next_operation_car.isin([3, 21, 81, 80])].index)
op_ind_as_set = set(op_ind)
intersection = list(op_ind_as_set.intersection(nop_ind))
data_duration = data_duration.drop(intersection)


# data_duration[['fin_operation_date',
#              'next_operation_car',
#              'next_operation_name',
#              'operation_duration']] = data_duration[['fin_operation_date',
#                                                      'next_operation_car',
#                                                      'next_operation_name',
#                                                      'operation_duration']].fillna(0)

In [673]:
# ноль минут операция скорее всего из-за человеческой ошибка
data_duration = data_duration[data_duration['operation_duration'] > 0]

In [674]:
# data_duration[data_duration['operation_duration'] == 8]['operation_name'].value_counts()

In [675]:
Q1 =  data_duration['operation_duration'].quantile(0.25)
Q3 = data_duration['operation_duration'].quantile(0.75)
IQR = Q3 - Q1
data_duration = data_duration[data_duration['operation_duration'] < Q3+1.5*IQR]

In [676]:
data_duration_short = data_duration[['operation_car', 'next_operation_car', 'operation_duration']]
data_duration_short = data_duration_short.reset_index(drop=True)

In [679]:
# start_time = time. time()
# создадим ключ для групировки
data_duration_short['event'] = 0
for i in range(data_duration_short.shape[0]):
    data_duration_short['event'][i] = str(int(data_duration_short.operation_car[i]))+'-'+str(int(data_duration_short.next_operation_car[i]))

# print("--- %s seconds ---" % (time. time() - start_time))
data_duration_short['ones'] = 1
data_duration_short


,operation_car,next_operation_car,operation_duration,event,ones
0,18.0,4.0,74.0,18-4,1
1,18.0,4.0,72.0,18-4,1
2,3.0,80.0,19.0,3-80,1
3,80.0,21.0,109.0,80-21,1
4,21.0,81.0,6.0,21-81,1
...,...,...,...,...,...
141423,81.0,80.0,3.0,81-80,1
141424,81.0,80.0,3.0,81-80,1
141425,81.0,80.0,3.0,81-80,1
141426,81.0,80.0,3.0,81-80,1


In [681]:
data_duration_group  = data_duration_short.groupby('event', as_index=False).agg({'operation_duration':'mean',
                                          'ones':'sum'})
data_duration_group['%'] = (data_duration_group['ones'] / data_duration_group.ones.sum())
data_duration_group

,event,operation_duration,ones,%
0,10-20,96.000000,1,0.000007
1,10-4,530.857143,49,0.000346
2,10-81,81.031838,691,0.004886
3,18-4,155.107368,475,0.003359
4,18-80,611.500000,4,0.000028
5,19-81,26.000000,1,0.000007
6,20-10,613.417647,170,0.001202
7,20-4,900.000000,2,0.000014
8,20-80,351.144928,69,0.000488
9,20-81,446.028974,2554,0.018059


In [682]:
data_duration_group['operation_car'] = data_duration_group['event'].str.split('-').str[0]
data_duration_group['next_operation_car'] = data_duration_group['event'].str.split('-').str[1]
data_duration_group

,event,operation_duration,ones,%,operation_car,next_operation_car
0,10-20,96.000000,1,0.000007,10,20
1,10-4,530.857143,49,0.000346,10,4
2,10-81,81.031838,691,0.004886,10,81
3,18-4,155.107368,475,0.003359,18,4
4,18-80,611.500000,4,0.000028,18,80
5,19-81,26.000000,1,0.000007,19,81
6,20-10,613.417647,170,0.001202,20,10
7,20-4,900.000000,2,0.000014,20,4
8,20-80,351.144928,69,0.000488,20,80
9,20-81,446.028974,2554,0.018059,20,81


In [683]:
print("Выберите операцию из списка ", list(data_duration_group.operation_car.unique()))
operation_car_ = input()
result = data_duration_group[data_duration_group['operation_car'] == operation_car_].sort_values('%', ascending=False).reset_index(drop=True)

print('вероятно следующее событие будет №: ', result['next_operation_car'][0])
print('вероятность: ', round(result['%'][0]), ' %')
print('предполагаемое время на операцию: ', round(result['operation_duration'][0]), ' минут')
result[:1]

Выберите операцию из списка  ['10', '18', '19', '20', '21', '3', '4', '80', '81']
10
вероятно следующее событие будет №:  81
вероятность:  0  %
предполагаемое время на операцию:  81  минут


,event,operation_duration,ones,%,operation_car,next_operation_car
0,10-81,81.031838,691,0.004886,10,81


In [684]:
data_duration_group['operation_car'] = data_duration_group['operation_car'].astype(int)
data_duration_group['next_operation_car'] = data_duration_group['next_operation_car'].astype(int)

In [685]:
# добавим описание операции
codes = pd.read_excel('Расшифровка операций с вагонами.xlsx')
codes.columns = ['next_operation_car', 'short_name', 'operation_name']
data_duration_group2 = data_duration_group.merge(codes, on='next_operation_car', how='left')
data_duration_group2

,event,operation_duration,ones,%,operation_car,next_operation_car,short_name,operation_name
0,10-20,96.000000,1,0.000007,10,20,ВЫГ1,ВЫГРУЗКА НА МЕСТАХ ОБЩ.ПОЛЬЗОВАНИЯ
1,10-4,530.857143,49,0.000346,10,4,ВКЛП,ВКЛЮЧЕНИЕ В ПОЕЗД
2,10-81,81.031838,691,0.004886,10,81,ПВПП,ПРИЕМ ВАГОНА С ПОД'ЕЗДНОГО ПУТИ
3,18-4,155.107368,475,0.003359,18,4,ВКЛП,ВКЛЮЧЕНИЕ В ПОЕЗД
4,18-80,611.500000,4,0.000028,18,80,СВПП,СДАЧА ВАГОНА НА ПОД'ЕЗДНОЙ ПУТЬ
5,19-81,26.000000,1,0.000007,19,81,ПВПП,ПРИЕМ ВАГОНА С ПОД'ЕЗДНОГО ПУТИ
6,20-10,613.417647,170,0.001202,20,10,ПГР1,ПОГРУЗКА НА МЕСТАХ ОБЩ.ПОЛЬЗОВАНИЯ
7,20-4,900.000000,2,0.000014,20,4,ВКЛП,ВКЛЮЧЕНИЕ В ПОЕЗД
8,20-80,351.144928,69,0.000488,20,80,СВПП,СДАЧА ВАГОНА НА ПОД'ЕЗДНОЙ ПУТЬ
9,20-81,446.028974,2554,0.018059,20,81,ПВПП,ПРИЕМ ВАГОНА С ПОД'ЕЗДНОГО ПУТИ


4.0 ВКЛЮЧЕНИЕ В ПОЕЗД или 0 означает что вагон покидает станцию

# прогноз по цепочке

берем операцию

узнаем след операцию, если она не 4 и не 0 - продолжаем

берем предыдущую операцию

узнаем след операцию, если она не 4 и не 0 - продолжаем

['10', '18', '19', '20', '21', '3', '4', '80', '81']

In [699]:
print("Выберите операцию из списка ", list(data_duration_group.operation_car.unique()))
operation_car_ = int(input())
time = 0
probability = 1

while operation_car_!=4 and operation_car_!=0:
    print('Начальное событие: ', operation_car_)
    print('-----')
    operation_car_
    result = data_duration_group[data_duration_group['operation_car'] == operation_car_].sort_values('%', ascending=False).reset_index(drop=True)
    result_operation = result['next_operation_car'][0]
    probability_operation = result['%'][0]
    time_operation = round(result['operation_duration'][0])
    
    operation_car_ = result_operation
    probability = probability * probability_operation
    time = time + time_operation
    print('вероятно следующее событие будет №: ', operation_car_)
    print('вероятность: ', probability, ' %')
    print('предполагаемое время на операцию: ', time, ' минут')
    print('-----')

Выберите операцию из списка  [10, 18, 19, 20, 21, 3, 4, 80, 81]
80
Начальное событие:  80
-----
вероятно следующее событие будет №:  21
вероятность:  0.2239938343185225  %
предполагаемое время на операцию:  302  минут
-----
Начальное событие:  21
-----
вероятно следующее событие будет №:  81
вероятность:  0.05249192275093186  %
предполагаемое время на операцию:  366  минут
-----
Начальное событие:  81
-----
вероятно следующее событие будет №:  4
вероятность:  0.011576371514845469  %
предполагаемое время на операцию:  758  минут
-----


In [632]:
data_duration_group

,event,operation_duration,ones,%,operation_car,next_operation_car
0,10-20,96.000000,1,0.000707,10,20
1,10-4,530.857143,49,0.034647,10,4
2,10-81,81.031838,691,0.488588,10,81
3,18-4,155.107368,475,0.335860,18,4
4,18-80,611.500000,4,0.002828,18,80
5,19-81,26.000000,1,0.000707,19,81
6,20-10,613.417647,170,0.120203,20,10
7,20-4,900.000000,2,0.001414,20,4
8,20-80,351.144928,69,0.048788,20,80
9,20-81,446.028974,2554,1.805866,20,81
